# Analysis of datasets - source: MagicDraw model

Mission: verify the integrity of a system model

Objectives: 
- use pattern matching to detect the inconsistencies,
- use graph analysis to display correction suggestions,
- allocate the correction of the system model to a user.

Note: the RDF dataset shall be 
- renamed: *.trig
- available through a query endpoint

In [1]:
# import necessary libraries 
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import Javascript

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

import networkx as nx
import snap

In [2]:
# definition of a function: sparql query
def query_fusekiData(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    sparql = SPARQLWrapper(sparql_service_url)
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [3]:
# definition of ipywidget 'Text' to prompt the  endpoint to query
layout = widgets.Layout(width='auto', height='40px') #set width and height of the different widgets
endpoint=widgets.Text(
    value='http://localhost:3030/*',
    placeholder='http://localhost:3030/',
    description='Endpoint: ',
    layout = layout,
    disabled=False
)
display(endpoint)

Text(value='http://localhost:3030/*', description='Endpoint: ', layout=Layout(height='40px', width='auto'), pl…

In [4]:
# definition of ipywidget 'Button' to take into account the endpoint value prompted
output = widgets.Output()
def on_button_clicked(b):
    with output:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        
b = widgets.Button(
    description='Analyze dataset', 
    layout = layout,
    disabled=False)
display(b)
b.on_click(on_button_clicked)

Button(description='Analyze dataset', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

In [6]:
# verification of the endpoint URL
req = Request(endpoint.value)
try:
    response = urlopen(req)
except HTTPError as e:
    print('Error - Endpoint: ' +endpoint.value + ' not found. Verify URL. (Error code: ', e.code, ')')
    raise SystemExit
except URLError as e:
    print('Error - Server not reached. (Reason: ', e.reason, ')')
    raise SystemExit
else:
    print ('Endpoint: ' +endpoint.value + ' found. Processing results now...')

Endpoint: http://localhost:3030/MDv20191202 found. Processing results now...


# Rule #1: Structural decomposition graph shall be directed, acyclic, connected and rooted

In [7]:
# composition sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?isContainedIn
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?isContainedIn }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [8]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No physical component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'isContainedIn.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [9]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'isContainedIn', create_using=nx.DiGraph())

In [10]:
# is graph directed
print("Graph is directed:", nx.is_directed(objGraph))

Graph is directed: True


In [11]:
# is graph acyclic
print("Graph is acyclic:", nx.is_directed_acyclic_graph(objGraph))
if not nx.is_directed_acyclic_graph(objGraph):
    print("Number of cycles:", len(list(nx.simple_cycles(objGraph))))
    print("Cycles found:")
    for el in list(nx.simple_cycles(objGraph)):
        print('- ', el)
    print("Among them, number of self-loop edges:", objGraph.number_of_selfloops())
    print("Nodes presenting a self-loop edge:" )
    for el in objGraph.nodes_with_selfloops():
        print('- ', el)
    print("\nSuggestion: Correct the graph so that it becomes acyclic.")

Graph is acyclic: True


In [12]:
# is graph connected
print("Graph is connected:", nx.is_weakly_connected(objGraph))
if not nx.is_weakly_connected(objGraph):
    print("Number of isolated nodes:", len(list(nx.isolates(objGraph))))
    print("Isolated nodes:")
    for el in list(nx.isolates(objGraph)):
        print('- ', el)
    print("\nSuggestion: Connect the isolated nodes.")

Graph is connected: True


In [13]:
# is graph rooted
obj = pd.DataFrame(objGraph.out_degree())
obj = obj[obj[1]==0]
print("Graph is rooted:", obj.shape[0] == 1)
if obj.shape[0] == 0:
    print("No roots found.")
    print("\nSuggestion: Correct the graph so that it presents one and only one root.")
if obj.shape[0] > 1:
    print("Number of roots found:", obj.shape[0])
    print("Roots found:")
    for el in obj[0]:
        print('-', el)
    print("\nSuggestion: Correct the graph so that it presents one and only one root.")

Graph is rooted: True


# Rule #2: Each Physical Component shall be contained in exactly one Subsystem

In [14]:
# aggregation sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?subSystem
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?subSystem }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [15]:
if 'subSystem.value' not in result_table.columns:
    print('Error - No subsystems found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'subSystem.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [16]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'subSystem', create_using=nx.DiGraph())

### a/ Is there Physical Component not contained in any Subystem?

In [17]:
sol_a = obj.subSystem.isnull().sum() # select rows based on condition 

print('Number of physical component not aggregated in any subsystem: '+str(sol_a))
sol_a = obj[obj['subSystem'].isnull()] 
sol_a = sol_a['physicalComponent'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical component currently not aggregated in any subsystem: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these physical components to one subsystem.")

Number of physical component not aggregated in any subsystem: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there any Physical Component contained in more than one Subsystem?

In [18]:
sol_b = obj.groupby(["physicalComponent"]).size()
sol_b = sol_b.to_frame("subSystem_count")

sol_b = sol_b[sol_b['subSystem_count'] > 1] # select rows based on condition 
print('Number of physical components aggregated in more than one subsystem: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical components currently aggregated in more than one subsystem: ')
    for i in range(sol_b.shape[0]):
        print('- Physical component ' +list(sol_b.index)[i] +' is aggregated in ' +str(list(sol_b.subSystem_count)[i]) +' different SubSystems: ')
        for j in range((obj[obj['physicalComponent'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['physicalComponent'] == list(sol_b.index)[i]].subSystem)[j])
    print("\nSuggestion: Look for these physical components and delete the extra aggregations.")

Number of physical components aggregated in more than one subsystem: 0
The model complies with the rule. No action needs to be performed.


# Rule #3: Each Subsystem shall be supplied by exactly one WorkPackage

In [19]:
# workpackage sparql query
query = """
PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?subsystem ?workPackage

FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#Subsystem_ID__18_5_3_8c20287_1572979614280_430250_25878>;
     rdfs:label ?subsystem.
  
  optional {
    ?b rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty project:supplies ; owl:someValuesFrom ?a ] ;
                                                                                rdfs:label ?workPackage. }  
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [20]:
if 'subsystem.value' not in result_table.columns:
    print('Error - No subsystem found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[["subsystem.value", "workPackage.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

### a/ Is there subsytems not supplied by any WorkPackage?

In [21]:
sol_a = obj.workPackage.isnull().sum() # select rows based on condition 

print('Number of subsystems not supplied by any workpackage: '+str(sol_a))
sol_a = obj[obj['workPackage'].isnull()] 
sol_a = sol_a['subsystem'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently not supplied by any workpackage: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these subsystems to one workpackage.")

Number of subsystems not supplied by any workpackage: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there subystems supplied by more than one WorkPackage?

In [22]:
sol_b = obj.groupby(["subsystem"]).size()
sol_b = sol_b.to_frame("workPackage_count")

sol_b = sol_b[sol_b['workPackage_count'] > 1] # select rows based on condition 
print('Number of subsystems supplied by more than one workpackage: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently supplied by more than one workpackage: ')
    for i in range(sol_b.shape[0]):
        print('- Subsystem ' +list(sol_b.index)[i] +' has relationship(s) with ' +str(list(sol_b.workPackage_count)[i]) +' different WorkPackages: ')
        for j in range((obj[obj['subsystem'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['subsystem'] == list(sol_b.index)[i]].workPackage)[j])
    print("\nSuggestion: Look for these subsytems and delete the extra relationships.")

Number of subsystems supplied by more than one workpackage: 0
The model complies with the rule. No action needs to be performed.


# Rule #4: Each Physical Component shall perform at least one Function

In [23]:
# assemblies perform functions sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?physicalComponent ?isContainedIn ?function
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?isContainedIn }.
    
  optional {
    ?c rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?g ].
    ?g rdfs:label ?function
    }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [24]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No physical component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'isContainedIn.value', 'function.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [25]:
# Verify the graph of aggregates is directed and acyclic
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'isContainedIn', create_using=nx.DiGraph())
if not nx.is_directed_acyclic_graph(objGraph):
    print('Error - The graph built from the structural decomposition is not a directed acyclic graph.')
    print("\nSuggestion: Correct the structural decomposition.")
    raise SystemExit

In [26]:
# Remove the implied relationships from the analysis: remove the assemblies that aggregate others

##
obj_a = obj[['isContainedIn', 'function']]
obj_a = obj_a.dropna(axis = 0, subset = ['isContainedIn'])
obj_a = obj_a.drop_duplicates(subset = 'isContainedIn')
obj_a = obj_a.rename(columns={'isContainedIn':'physicalComponent'})

## List all the assemblies
obj_b = obj[['physicalComponent', 'function']]
obj_b = obj_b.drop_duplicates(subset = 'physicalComponent')

## List the assemblies that aggregate no childs
obj = pd.concat([obj_a, obj_b])
obj = obj.drop_duplicates(subset = 'physicalComponent', keep = False)

In [27]:
sol = obj.function.isnull().sum() # select rows based on condition 

print('Number of physical components not performing any function: '+str(sol))
sol = obj[obj['function'].isnull()] 
sol = sol['physicalComponent'].tolist()

if not sol:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently not supplied by any workpackage: ')
    for el in sol:
        print ("- " +el)
    print("\nSuggestion: Connect each of these subsystems to one workpackage.")

Number of physical components not performing any function: 0
The model complies with the rule. No action needs to be performed.


# Rule #5: Each Physical Component shall satisfy at least one non functional Requirement

# Rule #6: Each Function shall be performed by at least one Physical Component

In [28]:
# functions performed by assemblies sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?function ?physicalComponent 
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?b ].
  ?b rdfs:label ?function

  optional {
    ?c a owl:Class ;
       rdfs:label ?physicalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?c rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?b ]. 
  }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [29]:
if 'function.value' not in result_table.columns:
    print('Error - No function found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['function.value', 'physicalComponent.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [30]:
sol = obj.physicalComponent.isnull().sum() # select rows based on condition 

print('Number of functions not performed by any physical component: '+str(sol))
sol = obj[obj['physicalComponent'].isnull()] 
sol = sol['function'].tolist()

if not sol:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of functions currently not performed by any physical component: ')
    for el in sol:
        print ("- " +el)
    print("\nSuggestion: Connect each of these functions to at least one physical component.")

Number of functions not performed by any physical component: 0
The model complies with the rule. No action needs to be performed.


In [31]:
sol_a = obj.workPackage.isnull().sum() # select rows based on condition 

print('Number of subsystems not supplied by any workpackage: '+str(sol_a))
sol_a = obj[obj['workPackage'].isnull()] 
sol_a = sol_a['subsystem'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently not supplied by any workpackage: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these subsystems to one workpackage.")

AttributeError: 'DataFrame' object has no attribute 'workPackage'

# Rule #7: Each Requirement shall specify at least one Physical Component or how Physical Components perform the function¶

In [ ]:
# functions performed by assemblies sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?requirement ?refinesRequirement ?physicalComponent ?fonctionalComponent 
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf  mission:Requirement ;
     rdfs:label ?requirement.

  optional {
    ?c a owl:Class ;
       rdfs:label ?physicalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?a rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:specifies ; owl:someValuesFrom ?c ].  
  }
  optional{
    ?e a owl:Class ;
       rdfs:label ?fonctionalComponent ;
       rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.
    ?a rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:specifies ; owl:someValuesFrom ?x ]. 

    ?x rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:hasPerformsSource ; owl:someValuesFrom ?e ].
  }
  optional {
    ?z rdfs:subClassOf [ a owl:Restriction ; owl:onProperty mission:refines ; owl:someValuesFrom ?a ] ;
        rdfs:label ?refinesRequirement.
  }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [ ]:
if 'requirement.value' not in result_table.columns:
    print('Error - No requirement found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['requirement.value', 'refinesRequirement.value', 'physicalComponent.value', 'fonctionalComponent.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [ ]:
# Verify the graph of requirements is directed and acyclic
objGraph = nx.from_pandas_edgelist(obj, 'requirement', 'refinesRequirement', create_using=nx.DiGraph())
if not nx.is_directed_acyclic_graph(objGraph):
    print('Error - The graph built from the requirement decomposition is not a directed acyclic graph.')
    print("\nSuggestion: Correct the requirement decomposition.")
    raise SystemExit

In [ ]:
#######
# Remove the implied relationships from the analysis: remove the requirements that are refined by others

##
obj_a = obj[['requirement', 'refinesRequirement', 'physicalComponent', 'fonctionalComponent']]
obj_a = obj_a.dropna(axis = 0, subset = ['refinesRequirement'])
obj_a = obj_a.drop_duplicates(subset = 'requirement')

## List all the requirements
obj_b = obj[['requirement', 'refinesRequirement', 'physicalComponent', 'fonctionalComponent']]
obj_b = obj_b.drop_duplicates(subset = 'requirement')

## List the assemblies that aggregate no childs
obj = pd.concat([obj_a, obj_b])
obj = obj.drop_duplicates(subset = 'requirement', keep = False)
obj = obj[['requirement', 'physicalComponent', 'fonctionalComponent']]

In [ ]:
sol_a = obj.physicalComponent.isnull() & obj.fonctionalComponent.isnull() # select rows based on condition 
sol_a = list(sol_a)

print('Number of requirements that do not specify any physical component nor how physical components perform the function: '+str(sol_a.count(True)))
sol_a = obj[(obj['physicalComponent'].isnull()) & (obj['fonctionalComponent'].isnull())]
sol_a = sol_a['requirement'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of requirements that currently do not specify any physical component nor how physical components perform the function: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these requirements to at least one physical component or to how physical components perform the function.")